In [ ]:
# 安装依赖
!pip install -q transformers datasets peft accelerate bitsandbytes

from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
import torch
import math

# 加载模型和数据
model_name =  "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
dataset = load_dataset("json", data_files="/content/alpaca_cropped.json")

# 拆分训练集和验证集
dataset = dataset["train"].train_test_split(test_size=0.1)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]

# 加载 tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

# 格式化函数
def format_sample(example):
    prompt = f"### Instruction:\n{example['instruction']}\n\n### Input:\n{example['input']}\n\n### Response:\n{example['output']}"
    tokenized = tokenizer(prompt, padding="max_length", truncation=True, max_length=512)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

# 数据处理
tokenized_train = train_dataset.map(format_sample, remove_columns=train_dataset.column_names)
tokenized_eval = eval_dataset.map(format_sample, remove_columns=eval_dataset.column_names)

# 加载模型并准备 LoRA 训练
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto",
    trust_remote_code=True
)
model = prepare_model_for_kbit_training(model)

# LoRA 配置
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)
model = get_peft_model(model, lora_config)

# 训练参数（去除 evaluation_strategy）
training_args = TrainingArguments(
    output_dir="/content/tinyllama-lora",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    num_train_epochs=3,
    warmup_steps=100,
    logging_dir="./logs",
    logging_steps=50,
    save_strategy="epoch",
    fp16=True,
    gradient_checkpointing=True,
    seed=42
)

# 启动 Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
)

# 开始训练
trainer.train()

# 手动评估 perplexity
def evaluate_perplexity(model, eval_dataset):
    model.eval()
    losses = []
    for batch in eval_dataset:
        input_ids = torch.tensor(batch["input_ids"]).unsqueeze(0).to(model.device)
        labels = torch.tensor(batch["labels"]).unsqueeze(0).to(model.device)
        with torch.no_grad():
            outputs = model(input_ids=input_ids, labels=labels)
        loss = outputs.loss.item()
        losses.append(loss)
    avg_loss = sum(losses) / len(losses)
    perplexity = math.exp(avg_loss)
    return {"eval_loss": avg_loss, "perplexity": perplexity}

# 调用评估函数
metrics = evaluate_perplexity(model, tokenized_eval)
print(f"\nEval Loss: {metrics['eval_loss']:.4f}, Perplexity: {metrics['perplexity']:.2f}")


Map:   0%|          | 0/468 [00:00<?, ? examples/s]

Map:   0%|          | 0/52 [00:00<?, ? examples/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jack2005 (jack2005-south-china-university-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:457: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Step,Training Loss



Eval Loss: 1.5667, Perplexity: 4.79


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel, TaskType
import torch
import math
import os

# 检查检查点文件夹
checkpoint_path = "/content/tinyllama-lora/checkpoint-45"

# 加载模型和数据
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
dataset = load_dataset("json", data_files="/content/alpaca_cropped.json")

# 拆分训练集和验证集
dataset = dataset["train"].train_test_split(test_size=0.1)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]
print(f"训练集大小: {len(train_dataset)}，验证集大小: {len(eval_dataset)}")

# 加载 tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

# 格式化函数
def format_sample(example):
    prompt = f"### Instruction:\n{example['instruction']}\n\n### Input:\n{example['input']}\n\n### Response:\n{example['output']}"
    tokenized = tokenizer(prompt, padding="max_length", truncation=True, max_length=512)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

# 数据处理
tokenized_train = train_dataset.map(format_sample, remove_columns=train_dataset.column_names)
tokenized_eval = eval_dataset.map(format_sample, remove_columns=eval_dataset.column_names)

# 加载基础模型
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto",
    trust_remote_code=True
)

model = prepare_model_for_kbit_training(model)

# 从检查点加载 LoRA 适配器
model = PeftModel.from_pretrained(
    model,
    checkpoint_path,  # 直接从检查点加载 LoRA 适配器
    is_trainable=True  # 确保模型可继续训练
)

# 检查 LoRA 参数

# 训练参数
training_args = TrainingArguments(
    output_dir="/content/tinyllama-lora",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    learning_rate=5e-5,
    num_train_epochs=4,  # 增加轮数以继续训练
    warmup_steps=100,
    logging_dir="./logs",
    logging_steps=50,
    save_strategy="epoch",
    fp16=True,
    gradient_checkpointing=True,
    seed=42
)

# 启动 Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
)

trainer.train()

# 手动评估 perplexity
def evaluate_perplexity(model, eval_dataset):
    model.eval()
    losses = []
    for batch in eval_dataset:
        input_ids = torch.tensor(batch["input_ids"]).unsqueeze(0).to(model.device)
        labels = torch.tensor(batch["labels"]).unsqueeze(0).to(model.device)
        with torch.no_grad():
            outputs = model(input_ids=input_ids, labels=labels)
        loss = outputs.loss.item()
        losses.append(loss)
    avg_loss = sum(losses) / len(losses)
    perplexity = math.exp(avg_loss)
    return {"eval_loss": avg_loss, "perplexity": perplexity}

# 调用评估函数
metrics = evaluate_perplexity(model, tokenized_eval)
print(f"\n评估损失: {metrics['eval_loss']:.4f}, 困惑度: {metrics['perplexity']:.2f}")

训练集大小: 468，验证集大小: 52


Map:   0%|          | 0/468 [00:00<?, ? examples/s]

Map:   0%|          | 0/52 [00:00<?, ? examples/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
50,0.464200



评估损失: 1.1518, 困惑度: 3.16


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel, TaskType
import torch
import math
import os

# 检查检查点文件夹
checkpoint_path = "/content/tinyllama-lora/checkpoint-60"

# 加载模型和数据
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
dataset = load_dataset("json", data_files="/content/alpaca_cropped.json")

# 拆分训练集和验证集
dataset = dataset["train"].train_test_split(test_size=0.1)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]
print(f"训练集大小: {len(train_dataset)}，验证集大小: {len(eval_dataset)}")

# 加载 tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

# 格式化函数
def format_sample(example):
    prompt = f"### Instruction:\n{example['instruction']}\n\n### Input:\n{example['input']}\n\n### Response:\n{example['output']}"
    tokenized = tokenizer(prompt, padding="max_length", truncation=True, max_length=512)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

# 数据处理
tokenized_train = train_dataset.map(format_sample, remove_columns=train_dataset.column_names)
tokenized_eval = eval_dataset.map(format_sample, remove_columns=eval_dataset.column_names)

# 加载基础模型
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto",
    trust_remote_code=True
)

model = prepare_model_for_kbit_training(model)

# 从检查点加载 LoRA 适配器
model = PeftModel.from_pretrained(
    model,
    checkpoint_path,  # 直接从检查点加载 LoRA 适配器
    is_trainable=True  # 确保模型可继续训练
)

# 检查 LoRA 参数

# 训练参数
training_args = TrainingArguments(
    output_dir="/content/tinyllama-lora",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    learning_rate=2e-5,
    num_train_epochs=5,  # 增加轮数以继续训练
    warmup_steps=300,
    logging_dir="./logs",
    logging_steps=50,
    save_strategy="epoch",
    fp16=True,
    gradient_checkpointing=True,
    seed=42
)

# 启动 Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
)

trainer.train()

# 手动评估 perplexity
def evaluate_perplexity(model, eval_dataset):
    model.eval()
    losses = []
    for batch in eval_dataset:
        input_ids = torch.tensor(batch["input_ids"]).unsqueeze(0).to(model.device)
        labels = torch.tensor(batch["labels"]).unsqueeze(0).to(model.device)
        with torch.no_grad():
            outputs = model(input_ids=input_ids, labels=labels)
        loss = outputs.loss.item()
        losses.append(loss)
    avg_loss = sum(losses) / len(losses)
    perplexity = math.exp(avg_loss)
    return {"eval_loss": avg_loss, "perplexity": perplexity}

# 调用评估函数
metrics = evaluate_perplexity(model, tokenized_eval)
print(f"\n评估损失: {metrics['eval_loss']:.4f}, 困惑度: {metrics['perplexity']:.2f}")

训练集大小: 468，验证集大小: 52


Map:   0%|          | 0/468 [00:00<?, ? examples/s]

Map:   0%|          | 0/52 [00:00<?, ? examples/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss


Step,Training Loss
50,0.304300



评估损失: 1.1614, 困惑度: 3.19

评估损失: 1.1614, 困惑度: 3.19


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel, TaskType
import torch
import math
import os

# 检查检查点文件夹
checkpoint_path = "/content/tinyllama-lora/checkpoint-75"

# 加载模型和数据
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
dataset = load_dataset("json", data_files="/content/alpaca_cropped.json")

# 拆分训练集和验证集
dataset = dataset["train"].train_test_split(test_size=0.1)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]
print(f"训练集大小: {len(train_dataset)}，验证集大小: {len(eval_dataset)}")

# 加载 tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

# 格式化函数
def format_sample(example):
    prompt = f"### Instruction:\n{example['instruction']}\n\n### Input:\n{example['input']}\n\n### Response:\n{example['output']}"
    tokenized = tokenizer(prompt, padding="max_length", truncation=True, max_length=512)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

# 数据处理
tokenized_train = train_dataset.map(format_sample, remove_columns=train_dataset.column_names)
tokenized_eval = eval_dataset.map(format_sample, remove_columns=eval_dataset.column_names)

# 加载基础模型
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto",
    trust_remote_code=True
)

model = prepare_model_for_kbit_training(model)

# 从检查点加载 LoRA 适配器
model = PeftModel.from_pretrained(
    model,
    checkpoint_path,  # 直接从检查点加载 LoRA 适配器
    is_trainable=True  # 确保模型可继续训练
)

# 检查 LoRA 参数

# 训练参数
training_args = TrainingArguments(
    output_dir="/content/tinyllama-lora",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    learning_rate=5e-5,
    num_train_epochs=3,  # 增加轮数以继续训练
    warmup_steps=50,
    logging_dir="./logs",
    logging_steps=50,
    save_strategy="epoch",
    fp16=True,
    gradient_checkpointing=True,
    seed=42
)

# 启动 Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
)

trainer.train()

# 手动评估 perplexity
def evaluate_perplexity(model, eval_dataset):
    model.eval()
    losses = []
    for batch in eval_dataset:
        input_ids = torch.tensor(batch["input_ids"]).unsqueeze(0).to(model.device)
        labels = torch.tensor(batch["labels"]).unsqueeze(0).to(model.device)
        with torch.no_grad():
            outputs = model(input_ids=input_ids, labels=labels)
        loss = outputs.loss.item()
        losses.append(loss)
    avg_loss = sum(losses) / len(losses)
    perplexity = math.exp(avg_loss)
    return {"eval_loss": avg_loss, "perplexity": perplexity}

# 调用评估函数
metrics = evaluate_perplexity(model, tokenized_eval)
print(f"\n评估损失: {metrics['eval_loss']:.4f}, 困惑度: {metrics['perplexity']:.2f}")

训练集大小: 468，验证集大小: 52


Map:   0%|          | 0/468 [00:00<?, ? examples/s]

Map:   0%|          | 0/52 [00:00<?, ? examples/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss



评估损失: 1.2872, 困惑度: 3.62


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel, TaskType
import torch
import math
import os

# 检查检查点文件夹
checkpoint_path = "/content/tinyllama-lora/checkpoint-75"

# 加载模型和数据
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
dataset = load_dataset("json", data_files="/content/alpaca_cropped.json")

# 拆分训练集和验证集
dataset = dataset["train"].train_test_split(test_size=0.1)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]
print(f"训练集大小: {len(train_dataset)}，验证集大小: {len(eval_dataset)}")

# 加载 tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

# 格式化函数
def format_sample(example):
    prompt = f"### Instruction:\n{example['instruction']}\n\n### Input:\n{example['input']}\n\n### Response:\n{example['output']}"
    tokenized = tokenizer(prompt, padding="max_length", truncation=True, max_length=512)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

# 数据处理
tokenized_train = train_dataset.map(format_sample, remove_columns=train_dataset.column_names)
tokenized_eval = eval_dataset.map(format_sample, remove_columns=eval_dataset.column_names)

# 加载基础模型
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto",
    trust_remote_code=True
)

model = prepare_model_for_kbit_training(model)

# 从检查点加载 LoRA 适配器
model = PeftModel.from_pretrained(
    model,
    checkpoint_path,  # 直接从检查点加载 LoRA 适配器
    is_trainable=True  # 确保模型可继续训练
)

# 检查 LoRA 参数

# 训练参数
training_args = TrainingArguments(
    output_dir="/content/tinyllama-lora",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    learning_rate=3e-5,
    num_train_epochs=3,  # 增加轮数以继续训练
    warmup_steps=100,
    logging_dir="./logs",
    logging_steps=50,
    save_strategy="epoch",
    fp16=True,
    gradient_checkpointing=True,
    seed=42
)

# 启动 Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
)

trainer.train()

# 手动评估 perplexity
def evaluate_perplexity(model, eval_dataset):
    model.eval()
    losses = []
    for batch in eval_dataset:
        input_ids = torch.tensor(batch["input_ids"]).unsqueeze(0).to(model.device)
        labels = torch.tensor(batch["labels"]).unsqueeze(0).to(model.device)
        with torch.no_grad():
            outputs = model(input_ids=input_ids, labels=labels)
        loss = outputs.loss.item()
        losses.append(loss)
    avg_loss = sum(losses) / len(losses)
    perplexity = math.exp(avg_loss)
    return {"eval_loss": avg_loss, "perplexity": perplexity}

# 调用评估函数
metrics = evaluate_perplexity(model, tokenized_eval)
print(f"\n评估损失: {metrics['eval_loss']:.4f}, 困惑度: {metrics['perplexity']:.2f}")

训练集大小: 468，验证集大小: 52


Map:   0%|          | 0/468 [00:00<?, ? examples/s]

Map:   0%|          | 0/52 [00:00<?, ? examples/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss



评估损失: 1.2167, 困惑度: 3.38
